In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ipywidgets as wg
from networkx.drawing.nx_pydot import graphviz_layout

# Ising model function definitions

In [2]:
def energy_calc(lat):
    ''' Calculated total energy of the lattice.
    
        Parameters:
            lat: [N] numpy array containing only 1 and -1 values representing the spin configuration.
        
        Returns: dtype float
            Energy of the spin configuration'''
    
    a = lat[:N-1]*lat[1:]
    # To ensure periodic boundary conditions (or to have the chain) the last entry is multiplied with the first entry of the lattice array. 
    b = lat[0]*lat[N-1]
    # Energy:
    E = -1*(np.sum(a)+np.sum(b))
    return(E)

In [3]:
def energy_nn(lat,x):
    ''' Calculate nearest neighbour interaction energy of 1 spin in the lattice.
    
        Parameters:
            lat: dtype numpy array
                [N] numpy array containing only 1 and -1 values representing the spin configuration.
            
            x: dtype int
                The coordinate of the spin of interest
        
        Returns: dtype float
             nearest neighbour interaction energy of the selected spin '''
    
    s = lat[x]
    nn = lat[(x+1)%N] + lat[(x-1)%N] 
    E = -1*nn*s
    return(E)

In [4]:
def specific_heat(x):
    ''' Calculates specific heat.
        
        Parameters: 
            x: dtype numpy array
                One dimensional array containing the energy data (or equivalent quantity)
            
        Returns: dtype numpy array
            One dimensional array containing the specific heat data'''
    
    cv = (np.var(x, axis=0))/(T**2)
    return(cv)

In [5]:
def susceptiblity(mag):
    ''' Calculates specific heat.
        
        Parameters: 
            mag: dtype numpy array
                One dimensional array containing the magnetization data
            
        Returns: dtype numpy array
            One dimensional array containing the susceptibility data'''
    
    susc = (np.var(mag))/T
    return(susc)

In [6]:
def metropolis(lat,E,M):
    ''' Implement metropolis algorithm for a single spin flip.
    
        Parameters: 
            lat: dtype numpy array
                [N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    # Select random spin (propose trial lattice with probability 1/L):
    x = np.random.randint(0,N)
    trial_lat = np.copy(lat)
    trial_lat[x] *= -1
    # Calculate energy difference between the old and trial configurations:
    dE = energy_nn(trial_lat,x) - energy_nn(lat,x)
    # Implement second stage of the metropolis algorithm:
    if dE <= 0:
        E = E + dE
        M = M - lat[x] + trial_lat[x] # substract old spin value and add new spin value to the magnetisation
        lat[x] = trial_lat[x]
    else:
        r = np.random.uniform(0,1)
        #p=np.exp(-dE/tau)
        p = exp[0]
        if r < p:
            E = E + dE
            M = M - lat[x] + trial_lat[x] # substract old spin value and add new spin value to the magnetisation
            lat[x] = trial_lat[x]
    return(lat,E,M)

In [7]:
def Wolffbacktrack(x,cluster,visited):
    ''' Part of the Wolff algorithm, it checks whether a spin is already in the cluster'''
    if cluster[x]:
        return False
    if visited[x]:
        return True#False
    return True

def grow_cluster(lattice,x,ClusterSpin,E,M, cluster,visited):
    """ Starting from a randomly chosen spin, creates a cluster with the Wolff Algorithm.
        
        Parameters:
            lattice: dtype numpy array
                The [N] array representing the spin configuration
            
            x: dtype int
                The coordinate of the spin that is added to the cluster
                
            ClusterSpin: dtype int
                The spin value of the cluster
            
            E and M: dtype int or float
                The energy and magnetization of the lattice
            
            cluster: dtype boolean numpy array
                [N] boolean numpy array indicating which spins are in the cluster
                
            visited: dtype boolean numpy array
                [N] boolean numpy array indicating which spins have already been trialed by the algorithm
                
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization"""
    
    old_E = energy_nn(lattice,x)
    lattice[x] *=-1
    dE = energy_nn(lattice,x) - old_E
    E = E + dE
    M = M - lattice[x]*-1 +lattice[x]
    #cluster.append([x,y])
    cluster[x] = True
    if Wolffbacktrack((x+1)%N, cluster, visited):
        E, M = try_add(lattice,(x+1)%N, ClusterSpin,E, M, cluster, visited)
    if Wolffbacktrack((x-1)%N, cluster, visited):
        E, M = try_add(lattice,(x-1)%N, ClusterSpin,E, M, cluster, visited)
    return(lattice,E,M)

def try_add(lattice,x,ClusterSpin,E,M, cluster, visited):
    """Checks of neighbor belongs to cluster, if so then add neighbor to cluster with a certain probability"""
    
    s_help = lattice[x]
    #global tau
    if ((s_help<0) != (ClusterSpin<0)):
        r = np.random.uniform(0,1)
        if r < (1-A_wolff):
            lattice, E, M = grow_cluster(lattice, x, ClusterSpin, E, M, cluster, visited)
        else:
            visited[x] = True
    return(E,M)

def Wolff(lattice, E, M):
    ''' Implements the wolff algorithm for a single timestep.
        
        Parameters: 
            lattice: dtype numpy array
                [N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    x = np.random.randint(0,N)
    cluster = np.full((N), False, dtype=bool)
    visited = np.full((N), False, dtype=bool)
    clusterspin = lattice[x]*-1
    lattice, E, M = grow_cluster(lattice,x,clusterspin, E, M, cluster, visited)
    return(lattice,E,M)

# Correspondence function definitions

In [8]:
def network_energy_calc(N_edges, n_l):
    # n_l is the total number of possible cross-edges (possible nn interactions)
    nwEl = (2*N_edges)-n_l
    return(nwEl)

In [9]:
def number_of_RGsteps(n):
    tel = 0
    neG_baseline = n-1
    lat_size = n 
    while n > 1:
        if n%2 != 0:
            raise ValueError("%d is not a power of the blocking size 2"%(lat_size))
        tel += 1
        n = n/2
        neG_baseline += n
    return(tel, neG_baseline)

In [10]:
def RGlats_skeleton():
    """ Creates the 'bare' shells of the RGb graph. That is, a list of spin lattices of decreasing size in accordance with the 
        branching factor 2, that serve as 'skeleton lattices' that can be modified by the RGb procedure
        
        Returns: 
            lats: dtype list
                list of lattice arrays. They serve as the initial spin lattices that constitute the shells of the RGb graph
    """
    lats = []; lats.append(np.zeros(N))
    n_nodes_shell = N
    while n_nodes_shell > 1:
        n_nodes_shell //= 2
        new_lat = np.zeros(n_nodes_shell)
        lats.append(new_lat)
    return(lats)     

In [11]:
def RGgraph_skeleton():
    """ Creates boundary graph (collection of nodes) corresponding to the spin lattice, and the 'bare' tree graph
        
        Returns:
            bdG: dtype networkx multigraph
                boundary graph
            
            bulkG: dtype networkx multigraph
                'bare' tree graph, i.e. the boundary + bulk tree graph that serves as a skeleton for the RGb graph
    """
    # Boundary graph
    bdG = nx.MultiGraph()
    for i in range(N):
        bdG.add_node(i)
    # Bulk graph
    n_nodes_shell = N; add = 0
    bulkG = bdG.copy()
    while n_nodes_shell > 1:
        for k in np.arange(0,n_nodes_shell,2):
            n_Gr = nx.number_of_nodes(bulkG); bulkG.add_node(n_Gr)
            bulkG.add_edge(n_Gr,k+add); bulkG.add_edge(n_Gr,k+1+add)
        add += n_nodes_shell
        n_nodes_shell //= 2
    return(bdG, bulkG)

In [12]:
def create_frozen_map(lat):
    """ Creates a boolean map of (freezes) all bonds (nn interactions) in the lattice
    
        Input:
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
        Returns:
            f_map: dtype numpy array
                [N] boolean array containing True if bond is present or False if not. For example spin lattice [1, -1, 1, 1]
                corresponds to f_map [True, True, False, False]
    """
    lat = np.array(lat)
    a = lat[1:]-lat[0:len(lat)-1]; b = lat[0]-lat[len(lat)-1]
    c = np.append(a,b)
    f_map = (c == 0)
    return(f_map)

In [13]:
def construct_graph(lat, lats, Gr):
    """ Construct graph from spin lattice according to the RGb procedure
        
        Input: 
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
            lats: dtype list
                list containing the initial spin lattices that constitutes the shells of the graph (i.e. as constructed by the 
                function 'RGlats_skeleton')
            
            Gr: dtype networkx multigraph
                'bare' tree graph that serves as the 'skeleton' for the RGb graph (i.e. as constructed by the function 
                'RGgraph_skeleton')
            
        Returns: 
            Gr: dtype networkx multigraph
                RGb graph
    """
    n_nodes_shell = N; shell = 0
    add = 0 
    Gr = Gr.copy()
    while shell < len(lats):
        lats[shell] = lat
        if shell == (len(lats)-1):
            break
            
        # Update RG graph accordingly:
        frm = create_frozen_map(lat)
        idx = np.where(frm==False)
        for i in idx[0]:
            Gr.add_edge(i+add, (i+1)%n_nodes_shell+add)    
        add += n_nodes_shell
        
        # Do RG step:
        old_lat = lat
        n_nodes_shell //= 2
        lat = lat.reshape(n_nodes_shell, 2)
        lat = np.sum(lat, axis=1)
        # Deal with ties:
        idx_zeros = np.where(lat==0)
        old_idx_zeros = tuple([2*i for i in idx_zeros])
        lat[idx_zeros] = old_lat[old_idx_zeros] #np.random.choice([1,-1], size=len(idx_zeros[0]))
        lat = lat/abs(lat)
        
        shell += 1
    return(Gr)

# Exhibition

In [14]:
def exhibition(temp, save):
    t_eq = 500
    t_sim = 10
    global T
    T = temp
    #global exp
    #exp = np.array([np.exp(-4/T),1,np.exp(4/T)]) # Store possible values of the acceptance probability (only need first value for metropolis)
    global A_wolff
    A_wolff = np.exp(-2/T) # Needed for acceptance value for Wolff
    
    # Check if N is a power of the block size
    nRGsteps, neGbase = number_of_RGsteps(N)
    
    # Initialize lattice
    lattice = -1*np.ones(N) 
    #lattice = np.random.choice([1,-1],size=(N))

    # Initialize figure
    %matplotlib notebook
    fig = plt.figure(figsize=(10,5)); plt.axis('off'); plt.title('N=%d, T=%.2f'%(N,temp))
    ax1 = fig.add_subplot(1,2,1); ax2 = fig.add_subplot(1,2,2)
    ax1.set_axis_off()
    
    # Initialize RG lattices
    lattices = RGlats_skeleton()

    # Initialize boundary and bulk graph
    bdG, bulkG = RGgraph_skeleton()
    
    # Calculate energy and magnetisation by performing a sweep over the whole lattice (only once, as at each step it is updated)
    E = energy_calc(lattice); M = np.sum(lattice) 

    for j in range(t_eq+t_sim):
        # Perform one MCS using metropolis algorithm:
        #for k in range(N):
            #lattice, E, M = metropolis(lattice,E,M)
            
        # Perform cluster flip using wolff algorithm:
        lattice, E, M = Wolff(lattice,E,M)  
        
        if j>t_eq: 
            # Draw boundary graph
            nx.draw_networkx(bdG,pos=nx.circular_layout(bdG),with_labels=True,node_color=lattice,ax=ax1,font_size=10)#,cmap=plt.cm.GnBu,vmin=-2,vmax=2)
        
            # Draw bulk graph
            ax2.clear(); ax2.set_axis_off()
            graph = construct_graph(lattice, lattices, bulkG)
            pos=graphviz_layout(graph,prog='twopi', root='%d'%(neGbase))
            nx.draw_networkx(graph,pos=pos,with_labels=True,ax=ax2,node_size=20,font_size=8,node_color='r')
    
            fig.canvas.draw()
            #time.sleep(1)
        
    if save:
        plt.savefig('RG1DcorrespondenceN=%dT=%d'%(N,temp))
    #nx.write_gml(graph,'howtree.gml')

In [15]:
N = 64
wg.interact_manual(exhibition, temp=(0.1,10), save=False)

interactive(children=(FloatSlider(value=5.05, description='temp', max=10.0, min=0.1), Checkbox(value=False, de…

<function __main__.exhibition(temp, save)>